In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import Span
from shapely.geometry import Point
import geopandas as gpd
import glob
from datetime import datetime
import bokeh
from bokeh.layouts import Row, column, gridplot
from bokeh.models import Title, Legend, TapTool, Range1d, TabPanel, Tabs
import matplotlib as mpl

from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)

import dask.dataframe as dd
from sklearn.preprocessing import StandardScaler
import dask_geopandas as dg


output_notebook()


Loading BokehJS ...

In [2]:
color_palette = [  '#4E79A7',  # Blue
    '#F28E2B',  # Orange
    '#E15759',  # Red
    '#76B7B2',  # Teal
    '#59A14F',  # Green
    '#EDC948',  # Yellow
    '#B07AA1',  # Purple
    '#FF9DA7',  # Pink
    '#9C755F',  # Brown
    '#BAB0AC',  # Gray
    '#7C7C7C',  # Dark gray
    '#6B4C9A',  # Violet
    '#D55E00',  # Orange-red
    '#CC61B0',  # Magenta
    '#0072B2',  # Bright blue
    '#329262',  # Peacock green
    '#9E5B5A',  # Brick red
    '#636363',  # Medium gray
    '#CD9C00',  # Gold
    '#5D69B1',  # Medium blue
]

# Ookla's Number of Speedtest Intelligence Users

Ookla Speedtests are a way for users to find out the download speed, upload speed and latency of their internet connection. This is a dataset of the number of users who took the speedtest. To account for change, we take the baseline number of users from November 2022 and show the percent change in daily users and change in weekly users. 

In [3]:
# extract all speedtest intelligence files
ios = pd.DataFrame()
stdesktop = pd.DataFrame()
android = pd.DataFrame()
stnet = pd.DataFrame()


android = dd.read_csv(glob.glob(f'../../data/ookla-speedtest/raw/android_*.csv'), low_memory=False, usecols = ['test_id', 'test_date', 'client_latitude', 'client_longitude', 'client_country'])
android['type'] = 'android'
ios = dd.read_csv(glob.glob(f'../../data/ookla-speedtest/raw/ios_*.csv'), low_memory=False, usecols = ['test_id', 'test_date', 'client_latitude', 'client_longitude', 'client_country'])
ios['type'] = 'ios'
stnet = dd.read_csv(glob.glob(f'../../data/ookla-speedtest/raw/stnet_*.csv'), low_memory=False, usecols = ['test_id', 'test_date', 'client_latitude', 'client_longitude', 'client_country'])
stnet['type'] = 'stnet'
stdesktop = dd.read_csv(glob.glob(f'../../data/ookla-speedtest/raw/stdesktop_*.csv'), low_memory=False, usecols = ['test_id', 'test_date', 'client_latitude', 'client_longitude', 'client_country'])
stdesktop['type'] = 'stdesktop'

In [4]:
# choose only the records where the records are from November 2022 till date and are taken in Turkey
ddf = dd.concat([android, ios, stnet, stdesktop])
ddf = ddf[ddf['client_country']=='Turkey']
ddf['test_date'] = dd.to_datetime(ddf['test_date'])
meta = ('test_date', 'datetime64[ns]')
#df['test_date'] = df['test_date'].apply(lambda x: x + timedelta(hours=3), meta=meta)
ddf = ddf[(ddf['test_date']>'2022-11-01')&(ddf['test_date']<'2023-03-16')]

# create a column for the number of 
#ddf['user_count'] = 1
ddf['date'] = ddf['test_date'].dt.date

In [5]:
# count the number of test_ids that were taken on a given day
ddf = ddf[['test_id', 'date', 'client_latitude', 'client_longitude', 'type']].groupby(['client_latitude','client_longitude', 'date', 'type']).count()[['test_id']].reset_index()

In [52]:
gddf = dg.from_dask_dataframe(
    ddf,
    geometry=dg.points_from_xy(ddf, "client_longitude", "client_latitude"),
).set_crs("EPSG:4326")

In [8]:
# read turkey shapefile and areas of interest
turkey_adm2 = gpd.read_file('../../data/turkey_administrativelevels0_1_2/tur_polbna_adm2.shp')
aoi = gpd.read_file('../../data/SYRTUR_tessellation.geojson')
affected_adm1 = list(turkey_adm2[turkey_adm2['adm1'].isin(aoi['ADM1_PCODE'].unique())]['adm1_en'].unique())
affected_adm2 = list(turkey_adm2[turkey_adm2['pcode'].isin(aoi['ADM2_PCODE'].unique())]['adm2_en'].unique())

In [53]:
gddf = gddf.sjoin(turkey_adm2)
gddf = gddf.drop(['client_latitude', 'client_longitude', 'OBJECTID', 'Shape_Leng', 'Shape_Area'], axis=1)

# add up all the test ids that were taken for that admin 2 level
gddf = gddf.groupby(['adm2_en', 'adm1_en', 'date']).sum().reset_index()

In [86]:
ooklaUsers = gddf.compute()
ooklaUsers['date'] = pd.to_datetime(ooklaUsers['date'])

In [87]:
ooklaUsers.rename(columns = {'test_id': 'n_users'}, inplace=True)
ooklaUsers = ooklaUsers[['adm1_en', 'adm2_en', 'date', 'n_users']]

In [88]:
ooklaUsers['date'] = pd.to_datetime(ooklaUsers['date'])
#ooklaUsers = ooklaUsers.groupby(['adm2_en', 'date']).sum('n_users').reset_index()
ooklaUsers = ooklaUsers[(ooklaUsers['date']>'2023-01-03')&(ooklaUsers['date']<'2023-03-13')]

In [106]:
baseline = ooklaUsers[ooklaUsers["date"].between("2022-11-01", "2023-11-30")]

In [107]:
scalers = {}

for adm2 in baseline["adm2_en"].unique():
    scaler = StandardScaler()
    scaler.fit(baseline[baseline["adm2_en"] == adm2][["n_users"]])

    scalers[adm2] = scaler

In [108]:
baseline = baseline.groupby(["adm2_en"]).agg({"n_users": ["mean", "std"]})
baseline.columns = baseline.columns.map(".".join)

In [109]:
ooklaUsersChange = ooklaUsers.merge(baseline, on = ['adm2_en'], how = 'left')

In [110]:
ooklaUsersChange['n_baseline'] = ooklaUsersChange['n_users.mean']
ooklaUsersChange["n_difference"] = ooklaUsersChange["n_users"] - ooklaUsersChange["n_baseline"]
ooklaUsersChange["percent_change"] = 100 * (ooklaUsersChange["n_users"] / (ooklaUsersChange["n_baseline"]) - 1)

In [111]:
for adm2, scaler in scalers.items():
    try:
        predicate = ooklaUsersChange["adm2_en"] == adm2
        activity = scaler.transform(ooklaUsersChange[predicate][["n_users"]])
        ooklaUsersChange.loc[predicate, "z_score"] = activity
    except:
        pass

In [112]:
#week['week_date'] = df['date'] - pd.to_timedelta(7, unit='d')

# #calculate sum of values, grouped by week
week = ooklaUsersChange.groupby([pd.Grouper(key='date', freq='W-MON'), 'adm2_en', 'adm1_en']).mean().reset_index()

#week['user_count'] = week['user_count'].apply(lambda x: round(x,0))

In [71]:
bokeh.core.validation.silence(EMPTY_LAYOUT, True)

def get_line_plot(ooklaUsers,title, source, earthquakes=False, subtitle=None, measure = 'percent_change'):

    p2 = figure(x_axis_type = 'datetime', width = 800, height = 400, toolbar_location='above')
    p2.add_layout(Legend(), "right")

    for id, adm2 in enumerate(ooklaUsers['adm2_en'].unique()):
        df = ooklaUsers[ooklaUsers['adm2_en']==adm2][['date', measure]].reset_index(drop=True)
        p2.line(df['date'], df[measure], line_width=2, line_color = color_palette[id], legend_label=adm2)

    p2.legend.click_policy='hide'
    if subtitle is not None:
        p2.title = subtitle


    title_fig = figure(title=title, toolbar_location=None,width=800, height=40, )
    title_fig.title.align = "left"
    title_fig.title.text_font_size = "20pt"
    title_fig.border_fill_alpha = 0
    title_fig.outline_line_width=0

    #with silence(MISSING_RENDERERS):
    sub_title = figure(title=source, toolbar_location=None,width=800, height=40, )
    sub_title.title.align = "left"
    sub_title.title.text_font_size = "10pt"
    sub_title.title.text_font_style="normal"
    sub_title.border_fill_alpha = 0
    sub_title.outline_line_width=0

    layout = column(title_fig, p2, sub_title)

    if earthquakes:
        p2.renderers.extend([
        Span(
            location=datetime(2023, 2, 6),
            dimension="height",
            line_color='#7C7C7C',
            line_width=2,
            line_dash=(4,4)
      ),
        Span(
            location=datetime(2023, 2, 20),
            dimension="height",
            line_color='#7C7C7C',
            line_width=2,
            line_dash=(4,4)
        ),
    ]
)

    return layout

In [113]:
tabs = []

for adm in affected_adm1:
    df = ooklaUsersChange[ooklaUsersChange['adm1_en']==adm] 

    tabs.append(
        TabPanel(
        child=get_line_plot(df, f"Number of daily users taking speedtest", "Source: Ookla Data for Good", earthquakes=True, subtitle = '% change compared to a 3 month prior baseline'),
                    title=adm.capitalize(),
                )
                )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)

In [114]:
tabs = []

for adm in affected_adm1:
    df = week[week['adm1_en']==adm] 

    tabs.append(
        TabPanel(
        child=get_line_plot(df, f"Weekly average users taking speedtest", "Source: Ookla Data for Good", earthquakes=True, subtitle = '% change compared to a 3 month prior baseline'),
                    title=adm.capitalize(),
                )
                )

tabs = Tabs(tabs=tabs, sizing_mode="scale_both")
show(tabs, warn_on_missing_glyphs=False)